In [ ]:
!pip install git+https://github.com/openai/whisper.git
!sudo apt update && sudo apt install ffmpeg
!pip install google-auth google-auth-oauthlib google-auth-httplib2 google-api-python-client whisper

import whisper
import sqlite3
import shutil
import os
import re


  Cloning https://github.com/openai/whisper.git to /tmp/pip-req-build-3blxnrig
  Running command git clone --filter=blob:none --quiet https://github.com/openai/whisper.git /tmp/pip-req-build-3blxnrig
  Resolved https://github.com/openai/whisper.git to commit ba3f3cd54b0e5b8ce1ab3de13e32122d0d5f98ab
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)


In [ ]:
model = whisper.load_model("base")

from google.oauth2 import service_account
from googleapiclient.discovery import build

from google.colab import auth
import pandas as pd
import os
import shutil
import time
from tqdm.notebook import tqdm
import numpy as np
from datetime import timedelta


auth.authenticate_user()
from google.auth import default
creds, _ = default()


100%|███████████████████████████████████████| 139M/139M [00:02<00:00, 63.3MiB/s]


In [ ]:
model = whisper.load_model("large")

100%|█████████████████████████████████████| 2.88G/2.88G [00:38<00:00, 79.9MiB/s]


In [ ]:
#test

def transcribe_video(file_path):
    try:
        result = model.transcribe(file_path, language='English')
        segments = result["segments"]
        for segment in segments:
            timestamp = segment['start']
            text = segment['text']
            formatted_transcription = f"{timestamp:.2f} : {text}\n"
            print(formatted_transcription)
    except Exception as e:
        print(f"An error occurred while transcribing {file_path}: {e}")

file_path = "/content/drive/MyDrive/COPY/VOL00001/NATIVES/NATIVE00001/CCRB-00001679.MOV"

transcribe_video(file_path)

[{'id': 0, 'seek': 0, 'start': 0.0, 'end': 6.0, 'text': ' Officer, we have a man down here.', 'tokens': [50365, 15434, 11, 321, 362, 257, 587, 760, 510, 13, 50665], 'temperature': 0.0, 'avg_logprob': -0.8807892481486003, 'compression_ratio': 1.5377358490566038, 'no_speech_prob': 0.47683173418045044}, {'id': 1, 'seek': 0, 'start': 6.0, 'end': 9.0, 'text': " He's got a knife and a knife and a knife.", 'tokens': [50665, 634, 311, 658, 257, 7976, 293, 257, 7976, 293, 257, 7976, 13, 50815], 'temperature': 0.0, 'avg_logprob': -0.8807892481486003, 'compression_ratio': 1.5377358490566038, 'no_speech_prob': 0.47683173418045044}, {'id': 2, 'seek': 0, 'start': 9.0, 'end': 11.0, 'text': " He's got a knife and a knife.", 'tokens': [50815, 634, 311, 658, 257, 7976, 293, 257, 7976, 13, 50915], 'temperature': 0.0, 'avg_logprob': -0.8807892481486003, 'compression_ratio': 1.5377358490566038, 'no_speech_prob': 0.47683173418045044}, {'id': 3, 'seek': 0, 'start': 11.0, 'end': 13.0, 'text': " I'm gonna unbe

In [ ]:
def transcribe_video(file_path):
    try:
        result = model.transcribe(file_path, language='English')
        segments = result["segments"]

        formatted_transcriptions = []
        for segment in tqdm(segments, desc="Transcribing video"):
            timestamp = segment['start']
            text = segment['text']
            text = re.sub(r'[^\x00-\x7F]+', '', text)
            td = timedelta(seconds=timestamp)
            hh_mm_ss_ssss = str(td)
            formatted_transcription = f"{hh_mm_ss_ssss} : {text}\n"
            formatted_transcriptions.append(formatted_transcription)

        return formatted_transcriptions
    except Exception as e:
        print(f"An error occurred while transcribing {file_path}: {e}")
        return []

def update_google_doc(doc_name, creds, transcriptions):
    try:
        service = build('docs', 'v1', credentials=creds)

        document = service.documents().create(body={"title": doc_name}).execute()
        document_id = document.get('documentId')

        for transcription in tqdm(transcriptions, desc="Updating Google Doc"):
            doc = service.documents().get(documentId=document_id).execute()
            doc_length = doc['body']['content'][-1]['endIndex'] - 1

            requests = [
                {
                    'insertText': {
                        'location': {
                            'index': doc_length,
                        },
                        'text': transcription
                    }
                }
            ]

            # update the document
            service.documents().batchUpdate(
                documentId=document_id, body={'requests': requests}).execute()

        print("Data successfully written to the Google Doc.")
    except Exception as e:
        print(f"An error occurred while writing to the Google Doc: {e}")

In [ ]:
file_path = "/content/drive/MyDrive/COPY/VOL00002/NATIVES/NATIVE00001/DEF_000229541.mp4"
doc_name = "Captain_Miller"
transcriptions = transcribe_video(file_path)
update_google_doc(doc_name, creds, transcriptions)

Transcribing video:   0%|          | 0/1850 [00:00<?, ?it/s]

Updating Google Doc:   0%|          | 0/1850 [00:00<?, ?it/s]

Data successfully written to the Google Doc.


In [ ]:
file_path = "/content/drive/MyDrive/COPY/VOL00002/NATIVES/NATIVE00001/DEF_000229545.mp4"
doc_name = "Chief_Monahan"
transcriptions = transcribe_video(file_path)
update_google_doc(doc_name, creds, transcriptions)

Transcribing video:   0%|          | 0/2650 [00:00<?, ?it/s]

Updating Google Doc:   0%|          | 0/2650 [00:00<?, ?it/s]

Data successfully written to the Google Doc.


In [ ]:
#Batch processing

from tqdm.notebook import tqdm
import numpy as np
import pandas as pd

transcriptions = []
video_paths = video_paths

results_dir = "/content/drive/MyDrive/Other/transcriptions"

if not os.path.exists(results_dir):
    os.makedirs(results_dir)

for video_path in tqdm(video_paths, desc="Transcribing videos"):
    segments = transcribe_video(video_path)
    for segment in segments:
        start_time = segment['start']
        end_time = segment['end']
        text = segment['text']
        file_name = os.path.basename(video_path)
        file_path = video_path
        transcriptions.append([
            file_name,
            file_path,
            start_time,
            end_time,
            text
        ])
    print(segment)

    # Save checkpoint after each file
    checkpoint_file = os.path.join(results_dir, 'transcriptions_checkpoint.csv')
    transcriptions_np = np.array(transcriptions)
    df = pd.DataFrame(transcriptions_np, columns=['File Name', 'File Path', 'Start Time', 'End Time', 'Transcription'])
    df.to_csv(checkpoint_file, index=False, encoding='utf-8')



In [ ]:
def transcribe_and_update_doc(file_path, doc_name, creds):
    try:
        service = build('docs', 'v1', credentials=creds)
        document = service.documents().create(body={"title": doc_name}).execute()
        document_id = document.get('documentId')

        # Transcribe the video and process segments incrementally
        result = model.transcribe(file_path, language='English')
        segments = result["segments"]

        for segment in tqdm(segments, desc="Transcribing and updating Google Doc"):
            timestamp = segment['start']
            text = segment['text']
            formatted_transcription = f"{timestamp:.2f} : {text}\n"

            requests = [
                {
                    'insertText': {
                        'location': {
                            'index': 1,
                        },
                        'text': formatted_transcription
                    }
                }
            ]

            # Execute the request to update the document
            service.documents().batchUpdate(
                documentId=document_id, body={'requests': requests}).execute()

            time.sleep(1)

        print("Data successfully written to the Google Doc.")
    except Exception as e:
        print(f"An error occurred while processing {file_path}: {e}")

creds, _ = default()

file_path = "/content/drive/MyDrive/COPY/VOL00002/NATIVES/NATIVE00001/DEF_000229541.mp4"
transcribe_and_update_doc(file_path, "Captain_Miller", creds)
